## CNN 

In [ ]:
from PyFunctions.Functions import *
from PyFunctions import var
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras import regularizers
import os
from tqdm import tqdm

In [ ]:
def get_conv_model_normal(dim):
    '''This function will compile and return a CNN neural network given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    
    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dropout(drop))

    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 




### Images

- Within the folder tests, add whatever image you would like that contains a gun and run the cell below

In [ ]:
def non_max_suppression(boxes, probs, overlapThresh=0.5):
    '''This function was taken and modified from PyImage search = a blog that teaches deep learning techniques using images. 
    This function will extract all the bounding boxes with a certain threshold of overlap'''
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)
    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return the indexes of only the bounding boxes to keep
    return pick

In [ ]:
def get_img_prediction_bounding_box(path, model, dim, edge = False):
    '''This function will create a bounding box over what it believes is a weapon given the image path, dimensions, and model used to detect the weapon.  Dimensions can be found within the Var.py file.  This function is still being used as I need to apply non-max suppresion to create only one bounding box'''
    
    img = get_image_value(path, dim, edge = edge)

    if edge == True:
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else: 
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = model.predict(img)[0]
    
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
        
    img = cv2.imread(path)
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process() 

    windows = []
    locations = []
    print(f'Creating Bounding Boxes for {path}')
    for x, y, w,h in rects: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        if edge == True:
            roi = get_edged(roi, dim = dim)
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)
    if edge == True:
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    else: 
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    windows = np.array(windows)
    locations = np.array(locations)
    predictions = model.predict(windows)
    
    pick = non_max_suppression(locations, probs = None)
    clone = img.copy() 
    clone2 = img.copy()
    for idx in pick: 
        startx, startx, endx, endy = locations[idx]
        cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        
    
#GOES HERE 
    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()


    return predictions

In [ ]:
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('ModelWeights/CNN-ModelCheckpointWeightsNoAugment.h5') #path to the model weights

test_folder = 'Tests/Photos'

for i in os.listdir(test_folder):
    if i == 'ipynb_checkpoints': 
        continue
    img_path = f'{test_folder}/{i}'
    predictions = get_img_prediction_bounding_box(img_path, normal_model, dim = var.norm_dimension, edge = edge)

### Video 
- within the folder Tests, add whatever video (.mp4) that contains a gun and run the cell below
- **Note**: Stick t video shorter than a minute or this process will take a very long time depending on your computer

In [ ]:
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('ModelWeights/CNN-ModelCheckpointWeightsNoAugment.h5')

images = get_vid_frames('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = True)

In [ ]:
pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]

In [ ]:
vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Tests/VideoOutput/BoundingBox.mp4',fourcc, 10, vid_dim) #change the filename to whatever you would like


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()